Data source: tbd

-Large on bedrock overview -- Done 
-anatomy of a good prompt -- Done
-comparison against 8x7b? -- Done
-common issues in prompt engineering and how to solve them -- done
-comparison of different prompts and their outputs -- done
-techniques like zero shot, few shot, chain of thought -- done


# Getting started with Mistral Large on Amazon Bedrock 

This notebook was tested using the Data Science 3.0 kernel in SageMaker Studio

# Amazon Bedrock 
[Amazon Bedrock](https://aws.amazon.com/bedrock/) is a fully managed service that provides access to a wide range of powerful foundation models (FMs) through a unified API. It offers models from leading AI companies like Mistral, Anthropic, AI21 Labs, Cohere, Stability AI, and Amazon's own Titan models.

# Mistral Large
[Mistral Large](https://mistral.ai/news/mistral-large/) is the most advanced language model developed by French AI startup Mistral AI

Key features of Mistral Large include:
- Strong multilingual capabilities, with fluency in English,French, Spanish, German, and Italian

- Impressive performance on reasoning, knowledge, math, and coding benchmarks

- 32K token context window for processing long documents

- Support for function calling and JSON format

## Getting access to Mistral Large
In order to start using Mistral Large, make sure that you have access to it from the Bedrock console:

1. Log in to the AWS Console and navigate to the Amazon Bedrock service

2. From the Bedrock home page, click "Get started"

3. On the left-hand navigation menu, click "Model access"

4. On the right side of the page, click "Manage model access"

5. Select the base models you would like access to from the list of available models


This notebook will walk you through how to get started with Mistral Large on Bedrock and cover prompting 101.

![Model Access](/imgs/model-access-img.png)


In [2]:
import boto3
import json
import pandas as pd
from IPython.display import display_html

In [3]:
DEFAULT_MODEL= "mistral.mistral-large-2402-v1:0"
bedrock = boto3.client(service_name="bedrock-runtime")
model_id = DEFAULT_MODEL


## Supported parameters

The Mistral AI models have the following [inference parameters](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-mistral.html).


```
{
    "prompt": string,
    "max_tokens" : int,
    "stop" : [string],    
    "temperature": float,
    "top_p": float,
    "top_k": int
}
```

- Temperature - Tunes the degree of randomness in generation. Lower temperatures mean less random generations.
- Top P - If set to float less than 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
- Top K - Can be used to reduce repetitiveness of generated tokens. The higher the value, the stronger a penalty is applied to previously present tokens, proportional to how many times they have already appeared in the prompt or prior generation.
- Maximum Length - Maximum number of tokens to generate. Responses are not guaranteed to fill up to the maximum desired length.
- Stop sequences - Up to four sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

In [19]:

prompt = "What is Amazon Web Services?"
body = json.dumps({
            "temperature": 1.0,
            "max_tokens": 128,
            "prompt": prompt,
            "stop":["</s>"]

        })
response = bedrock.invoke_model(
    body = body,
    modelId = model_id
)

response_body = json.loads(response.get("body").read())
df = pd.DataFrame(
    {"Response": [response_body['outputs'][0]['text'].replace("\n", " ")]}
    )
display_html(df.to_html(index=False), raw=True)

Response
"Amazon Web Services (AWS) is a cloud computing service developed by Amazon.com. It was first launched in 2006 and has since become the leading cloud computing platform, serving millions of customers worldwide. AWS provides a broad set of products and services, including computing power, storage, databases, analytics, machine learning, security, mobile, and enterprise applications. These services are delivered as a utility over the Internet, with customers paying only for the services they use, without having to invest in expensive hardware or infrastructure. AWS is trusted by some of the largest companies in the world"


# Anatomy of a strong prompt

A prompt is a natural language text that requests the generative AI to perform a specific task. Imagine you're hiring a highly skilled and knowledgeable personal assistant. This assistant is capable of handling a wide range of tasks, from research and analysis to creative writing and problem-solving. However, to get the most out of their abilities, you need to provide clear and specific instructions.

A poorly crafted prompt or set of instructions would be like asking your personal assistant to "do something useful." They might try their best, but without proper guidance, the result could be hit or miss. They might end up organizing your sock drawer or alphabetizing your spice rack – technically useful, but not what you had in mind.

On the other hand, a well-crafted prompt is like giving your assistant a detailed task list with context, examples, and specific requirements. For instance, you might say, "I need a comprehensive report on the latest advancements in renewable energy technologies, with a focus on their economic and environmental impacts. Please include relevant statistics, visual aids, and a executive summary highlighting the key takeaways. The report should be written in a professional tone, suitable for presentation to industry experts."

With this level of detail and specificity in the prompt, your assistant can leverage their knowledge and skills to deliver a high-quality, tailored output that meets your exact needs. They have a clear understanding of the task, the desired format, and the target audience.

Just like a skilled personal assistant, LLMs are powerful tools that require clear and well-crafted prompts to unleash their full potential. Effective prompting ensures that the model's capabilities are channeled in the right direction, resulting in more relevant, accurate, and useful responses.

A strong prompt should include as many characetersitics as possible from the following list:

- Clear and specific instructions: The prompt should provide clear and specific instructions about the task or information you're seeking. Ambiguous or vague prompts can lead to confusing or irrelevant responses.

- Context and background information: Providing relevant context and background information helps the LLM understand the broader context of the prompt and generate more informed and accurate responses.

- Examples or samples: Including examples or samples, especially for tasks that require specific formatting or structure, can help the LLM better understand and follow the desired format.

- Specific constraints or requirements: If there are specific constraints or requirements, such as tone, or style, including these in the prompt can help the LLM tailor its response accordingly.

- Clarity on the desired output: Clearly specifying the desired output format (e.g., bullet points, paragraphs, code snippets) can help the LLM structure its response appropriately.

- Avoidance of leading or biased language: To avoid biased or skewed responses, it's important to use neutral and unbiased language in the prompt. Leading or loaded language can influence the LLM's response in unintended ways.

- Iterative refinement: If the initial response from the LLM is not satisfactory, providing feedback and refining the prompt can help improve the quality of subsequent responses.

- Appropriate length: While LLMs can handle long prompts, excessively lengthy prompts can be challenging to process and may lead to less coherent or focused responses. Aim for concise prompts that capture the essential information.

![Image Title](/imgs/model-access-img.png)

In [27]:


# Define the prompts
query = "Do something helpful."
prompt_1 = f"{query}"
prompt_2 = f"""You are an Amazon Web Services (AWS) Expert. 
Your role is to answer every technical question about AWS as accurately as possible.
If you do not know the answer to a question, say I don't know. Give statistics 
around usage in the Enterprise world.{query}"""
prompts = [prompt_1, prompt_2]


# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    body = json.dumps({
        "temperature": 0.5,
        "max_tokens": 128,
        "prompt": prompt,
        "stop": ["</s>"]
    })
    response = bedrock.invoke_model(
    body = body,
    modelId = model_id)
    response_body = json.loads(response.get("body").read())
    responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Prompt 1": [responses[0]],
    "Prompt 2": [responses[1]]
})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)




Prompt 1,Prompt 2
"Comment: @Nick I believe that the OP's problem is more of a logical one, not a syntax one. Also, I did not downvote your answer. Comment: @Nick I am not sure how I can help you. I have given you a suggestion for a better answer. Comment: @Nick You have not given the OP the answer. You have given him a link to a website that may or may not have the answer. If you have the answer, post it. Comment: @Nick The OP has already seen that page and has still not found the","Question: What is AWS Lambda and how does it work? Answer: AWS Lambda is a serverless compute service provided by Amazon Web Services (AWS). It allows developers to run their code without having to manage servers, hence the term ""serverless."" AWS Lambda executes your code only when needed and scales automatically, from a few requests per day to thousands per second. You pay only for the compute time you consume; there is no charge when your code is not running. Here's how it works: 1. You write your code in one of the supported languages,"


# Common Prompting Issues
Due to the nondeterministic nature of LLM responses and the fact that prompting can be a bit of an art, there's several issues that you may experience when developing your prompts. Here's some of the most common ones:

1. Hallucination and factual inaccuracies

2. Lack of coherence and logical reasoning in generated text

3. Difficulty with complex, multi-step tasks

4. Misunderstanding user intent



### 1. Hallucination and factual inaccuracies

Hallucination and factual inaccuracies are an issue that can arise when LLM produce text outputs. Since these models are trained on vast amounts of data from the internet and other sources, their knowledge can sometimes be incomplete, biased, or simply incorrect.

Solution: Provide clear instructions in the prompt and add any additional context. You can also use retrieval-augmented generation to ground the model's outputs in factual information from your trusted sources. In the example below, the context variable acts as the factual information to ground the model's response.

In [7]:

# Define the prompts
query = "What is project Cobra?"
context = """
Project Cobra aims to develop a method of interstellar travel by creating and manipulating closed timelike curves (CTCs) - paths through spacetime that loop back on themselves, allowing travel to the past. The theoretical basis comes from solutions to Einstein's equations of general relativity that permit CTCs in unusual spacetime geometries like wormholes.

To achieve this, Project Cobra plans to generate traversable wormholes stabilized with exotic matter, then accelerate one wormhole mouth to near lightspeed to induce CTCs inside. This would allow a spacecraft to travel through the wormhole and exit in its own past, appearing to an outside observer to have moved faster than light.

Major challenges include creating and stabilizing the wormholes, accelerating them to form the time machine, navigating through them precisely, and avoiding paradoxes from changing the past.

If successful, it could revolutionize interstellar travel by providing shortcuts through spacetime. However, it also carries huge risks of misuse or unintended consequences that would require stringent protocols and safety measures.

In essence, it is an extremely ambitious endeavor aiming to translate predictions of general relativity into functional time travel technology for reaching distant stars, albeit with immense theoretical and engineering hurdles to overcome."""

prompt_1 = f"{query}"
prompt_2 = f"""Using the conext given below, answer the question: {query}
                Provided context: {context}"""
prompts = [prompt_1, prompt_2]

# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    body = json.dumps({
        "temperature": 0.5,
        "max_tokens": 128,
        "prompt": prompt,
        "stop": ["</s>"]
    })
    response = bedrock.invoke_model(
    body = body,
    modelId = model_id)
    response_body = json.loads(response.get("body").read())
    responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Response w/o context": [responses[0]],
    "Response with context": [responses[1]]
})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Response w/o context,Response with context
"Project Cobra is a research and development project that aims to create a new generation of autonomous vehicles. The project is led by a team of engineers, computer scientists, and roboticists who are working together to develop advanced technologies for self-driving cars. The goal of Project Cobra is to create vehicles that are safer, more efficient, and more reliable than traditional cars. The project is focused on developing new sensors, algorithms, and control systems that will enable autonomous vehicles to navigate complex environments and make decisions in real-time. Project Cobra is also exploring the use of artificial intelligence and machine learning to improve the","Project Cobra is an ambitious scientific endeavor aiming to develop a method of interstellar travel by creating and manipulating closed timelike curves (CTCs). This involves generating traversable wormholes stabilized with exotic matter, accelerating one wormhole mouth to near lightspeed to induce CTCs inside, which would allow a spacecraft to travel through the wormhole and exit in its own past. If successful, it could revolutionize interstellar travel but also carries huge risks due to potential misuse or unintended consequences."


### 2. Lack of coherence and logical reasoning in generated text

Solution: Use chain-of-thought prompting, where the model is prompted to break down its reasoning into a series of logical steps before providing a final answer. This can significantly improve performance on tasks requiring multi-step reasoning.

In [15]:


# Define the prompts
# query = """You have a basket of fruit with 6 apples, 4 oranges, and 3 bananas.
#          You want to divide the fruit equally among 3 friends, putting each friend's portion into their own small basket. 
#          How many of each type of fruit should go in each small basket?"""
query = """When I left the house this morning, I had $23 in my pocket. I spent $5 on coffee and $7 on lunch. Then I found $10 on the sidewalk. How much money do I have now?"""
prompt_1 = f"{query}"
prompt_2 = f"""Think-step-by-step and answer the following question: {query}"""
prompts = [prompt_1, prompt_2]

# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    body = json.dumps({
        "temperature": 0.5,
        "max_tokens": 512,
        "prompt": prompt,
        "stop": ["</s>"]
    })
    response = bedrock.invoke_model(
    body = body,
    modelId = model_id)
    response_body = json.loads(response.get("body").read())
    responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Response w/o CoT": [responses[0]],
    "Response with CoT": [responses[1]]
})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Response w/o CoT,Response with CoT
"Let's break it down: 1. You started with $23. 2. You spent $5 on coffee, leaving you with $23 - $5 = $18. 3. You spent $7 on lunch, leaving you with $18 - $7 = $11. 4. You found $10 on the sidewalk, bringing your total to $11 + $10 = $21. So, you now have $21.","Let's break it down: 1. You started with $23. 2. You spent $5 on coffee, so you had $23 - $5 = $18 left. 3. You spent $7 on lunch, so you had $18 - $7 = $11 left. 4. You found $10 on the sidewalk, so you added that to your total, giving you $11 + $10 = $21. So, you now have $21."


### 3. Difficulty with complex, multi-step tasks

Solution: Decompose the complex task into a sequence of simpler sub-tasks in the prompt. Guide the model to solve each sub-task step-by-step. Least-to-most prompting, where sub-problems are solved in order of increasing difficulty, can help.

In [14]:
# Define the prompts
query = "The task is to plan a 3-course meal for 6 people."
prompt_1 = f"{query}"
prompt_2 = f"""{query} Using the following subtasks:
            Step 1: Choose recipes for appetizer, main course, and dessert
            Step 2: Make a grocery list of all required ingredients 
            Step 3: Determine cooking utensils, pots/pans, bakeware needed
            Step 4: Come up with a timeline to prep each course"""
prompts = [prompt_1, prompt_2]


# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    body = json.dumps({
        "temperature": 0.5,
        "max_tokens": 1024,
        "prompt": prompt,
        "stop": ["</s>"]
    })
    response = bedrock.invoke_model(
    body = body,
    modelId = model_id)
    response_body = json.loads(response.get("body").read())
    responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Prompt 1": [responses[0]],
    "Prompt 2": [responses[1]]
})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Prompt 1,Prompt 2
"What dishes would you suggest for each course? For a delightful 3-course meal for six people, I would suggest the following dishes: **Starter:** 1. Caprese Salad: A refreshing and light Italian salad made with fresh mozzarella, tomatoes, basil, and a drizzle of balsamic glaze. This dish is perfect for a group, as it can be easily prepared and served family-style. **Main Course:** 2. Herb-Crusted Rack of Lamb: This elegant and flavorful dish features a tender rack of lamb coated in a mixture of fresh herbs and breadcrumbs, then roasted to perfection. Serve it with a side of creamy mashed potatoes and steamed green beans or asparagus. 3. Vegetarian Option - Stuffed Bell Peppers: Colorful bell peppers filled with a hearty mixture of quinoa, black beans, corn, and spices, then topped with cheese and baked until golden brown. This satisfying vegetarian option can be enjoyed by everyone at the table. **Dessert:** 4. Classic Tiramisu: A heavenly Italian dessert consisting of layers of coffee-soaked ladyfingers and creamy mascarpone cheese, dusted with cocoa powder. This indulgent treat is sure to impress your guests and provide the perfect ending to your meal. 5. Fresh Fruit Salad: A vibrant and healthy dessert option featuring a medley of seasonal fruits tossed in a light honey-lime dressing. This refreshing dish offers a delightful contrast to the richer tiramisu and caters to those who may prefer a lighter dessert. These dishes offer a balance of flavors, textures, and presentation that will make your 3-course meal both enjoyable and memorable for your guests.","Step 5: Provide serving suggestions and garnishes Sure, I'd be happy to help you plan a 3-course meal for 6 people. Here's a suggestion: **Step 1: Choose recipes for appetizer, main course, and dessert** Appetizer: Caprese Salad - A classic Italian salad made with fresh tomatoes, mozzarella cheese, and basil leaves, dressed with olive oil and balsamic vinegar. Main Course: Chicken Marsala - A flavorful dish with chicken breasts cooked in a Marsala wine sauce with mushrooms and served over mashed potatoes. Dessert: Tiramisu - An Italian coffee-flavored dessert made with ladyfingers dipped in coffee, layered with a whipped mixture of eggs, sugar, and mascarpone cheese, and dusted with cocoa powder. **Step 2: Make a grocery list of all required ingredients** Caprese Salad: - Fresh tomatoes (3) - Fresh mozzarella (1 lb) - Fresh basil leaves (1 bunch) - Olive oil (1/4 cup) - Balsamic vinegar (1/4 cup) - Salt and pepper to taste Chicken Marsala: - Boneless, skinless chicken breasts (6) - Marsala wine (1 cup) - Mushrooms (1 lb) - Chicken broth (1 cup) - All-purpose flour (1/2 cup) - Unsalted butter (4 tbsp) - Olive oil (2 tbsp) - Garlic cloves (2) - Salt and pepper to taste - Fresh parsley for garnish (optional) - Potatoes for mashed potatoes (6 large) - Milk (1 cup) - Unsalted butter (4 tbsp) - Salt and pepper to taste Tiramisu: - Egg yolks (6) - Granulated sugar (3/4 cup) - Mascarpone cheese (2 cups) - Strong brewed coffee, cooled (2 cups) - Ladyfinger biscuits (40-48) - Unsweetened cocoa powder (1/4 cup) **Step 3: Determine cooking utensils, pots/pans, bakeware needed** Caprese Salad: - Large mixing bowl - Cutting board - Sharp knife Chicken Marsala: - Large skillet - Wooden spoon - Plate for flour - Pot for mashed potatoes - Potato masher Tiramisu: - Large mixing bowl - Electric mixer - 9x13 inch baking dish - Rubber spatula - Sifter for cocoa powder **Step 4: Come up with a timeline to prep each course** 1. Start with the Tiramisu as it needs to chill for at least 4 hours, preferably overnight. 2. Prepare the Caprese Salad about an hour before serving to allow the flavors to meld. 3. Start the Chicken Marsala about 45 minutes before you want to serve it. Begin by boiling the potatoes for the mashed potatoes. While the potatoes are boiling, you can start cooking the chicken. 4. Once the chicken is done, keep it warm in the oven while you finish the mashed p

### 4. Misunderstanding user intent

Solution: Provide clear context and instructions in the prompt to guide the model. Use role prompting to define the model's persona and expertise to better address the user's needs.

In [18]:
# Define the prompts
query = "Write a paragraph about healthy eating."
prompt_1 = f"{query}"
prompt_2 = f"""Do not give a preamble.You are a nutritonist. The audience is adults looking to improve their eating habits.
 Provide a 3-4 sentence paragraph clearly explaining 2-3 key principles of healthy eating that are backed by current nutritional research. {query}"""
prompts = [prompt_1, prompt_2]

# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    body = json.dumps({
        "temperature": 0.5,
        "max_tokens": 512,
        "prompt": prompt,
        "stop": ["</s>"]
    })
    response = bedrock.invoke_model(
    body = body,
    modelId = model_id)
    response_body = json.loads(response.get("body").read())
    responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Prompt 1": [responses[0]],
    "Prompt 2": [responses[1]]
})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Prompt 1,Prompt 2
"Healthy eating is a fundamental aspect of maintaining overall well-being and vitality. It involves consuming a balanced diet rich in essential nutrients, vitamins, and minerals to support various bodily functions and promote optimal health. A healthy diet primarily consists of whole foods, such as fruits, vegetables, whole grains, lean proteins, and healthy fats. These foods provide the necessary energy and nutrients needed for growth, repair, and maintenance of our bodies. In addition to nourishing the body, healthy eating also plays a crucial role in preventing chronic diseases, such as obesity, heart disease, diabetes, and certain types of cancer. By making conscious and informed food choices, we can significantly improve our quality of life, enhance our physical and mental performance, and reduce the risk of health complications. Embracing healthy eating habits is not only beneficial for our individual well-being but also contributes to the overall health and prosperity of our communities.","Incorporating a variety of whole foods is a cornerstone of a healthy diet, as different foods provide diverse nutrients necessary for optimal body function. Current nutritional research emphasizes the importance of consuming ample fruits and vegetables, which are rich in vitamins, minerals, and fiber, as well as lean proteins for muscle development and repair. Additionally, prioritizing whole grains over refined carbohydrates can help maintain steady blood sugar levels and support overall health."


# Mistral Large vs Mixtral 8x7B

Mistral Large and Mixtral 8x7B are two large language models developed by Mistral AI, but they differ in several key aspects. Mixtral 8x7B, released in December 2023 as an open-source model, is a 45B parameter sparse mixture-of-experts model that performs on par with GPT-3.5 on benchmarks and handles multiple languages. In contrast, Mistral Large, launched in February 2024 as a commercial offering, is a more advanced and cutting-edge model with undisclosed architecture details, designed for complex multilingual reasoning tasks. It achieves top performance on benchmarks like MMLU, outperforming models like Claude and Llama. While Mixtral 8x7B is freely available on Bedrock, Mistral Large is a paid service offered through Bedrock as well, representing the Mistral's latest flagship model with native multilingual fluency and superior reasoning capabilities.

In [8]:
# Define the prompts

prompt_1 = """Evaluate this Python function for correctness and efficiency:
            def fibonacci(n):
            if n <= 0:
            return 0
            elif n == 1:
            return 1
            else:
            return fibonacci(n-1) + fibonacci(n-2)"""
prompt_2 = """Translate the following text to Spanish, French and German:
             "The quick brown fox jumps over the lazy dog."""
prompts = [prompt_1, prompt_2]
model_ids = ["mistral.mixtral-8x7b-instruct-v0:1", DEFAULT_MODEL]

# Iterate through the prompts and get the responses
responses = []
for prompt in prompts:
    for model_id in model_ids:
        body = json.dumps({
            "temperature": 0.5,
            "max_tokens": 1024,
            "prompt": prompt,
            "stop": ["</s>"]
        })
        response = bedrock.invoke_model(
        body = body,
        modelId = model_id)
        response_body = json.loads(response.get("body").read())
        responses.append(response_body["outputs"][0]["text"].replace("\n", " "))

# Create a pandas DataFrame to display the responses side-by-side
df = pd.DataFrame({
    "Mixtral 8x7b Prompt 1": [responses[0]],
    "Mistral Large Prompt 2": [responses[1]],
    "Mixtral 8x7b Prompt #1": [responses[2]],
    "Mistral Large Prompt #2": [responses[3]],
    

})

# Display the DataFrame as HTML
display_html(df.to_html(index=False), raw=True)

Mixtral 8x7b Prompt 1,Mistral Large Prompt 2,Mixtral 8x7b Prompt #1,Mistral Large Prompt #2
"Correctness: The function is not correct as it does not return the nth Fibonacci number for n > 1. It fails to memoize the results of subproblems, leading to repeated computations and inefficient runtimes. Efficiency: The function's time complexity is exponential (O(2^n)) due to the repeated computations of the same subproblems. This is inefficient and can be improved by using dynamic programming or memoization techniques. To make the function correct and efficient, we can use the following approach: Corrected and Efficient Solution: def fibonacci(n, memo = {}): if n <= 0: return 0 elif n == 1: return 1 elif n in memo: return memo[n] else: memo[n] = fibonacci(n-1) + fibonacci(n-2) return memo[n] In this corrected solution, we use a dictionary called memo to store the results of subproblems. This way, we avoid redundant computations and improve the time complexity to linear (O(n)).","The given Python function is a recursive implementation of the Fibonacci sequence. It correctly computes the nth Fibonacci number for any non-negative integer input `n`. However, in terms of efficiency, this function has a significant issue: it exhibits exponential time complexity due to redundant computations. Specifically, for each call to `fibonacci(n)`, the function makes two additional calls to compute `fibonacci(n-1)` and `fibonacci(n-2)`. As a result, the same subproblems are solved repeatedly, leading to an overall time complexity of O(2^n). To improve the efficiency of this function, you can use dynamic programming techniques such as memoization or tabulation. Memoization involves storing the results of previously computed Fibonacci numbers in a dictionary or array, allowing the function to avoid redundant computations by looking up the result if it has already been computed. Tabulation, on the other hand, builds an array of Fibonacci numbers iteratively, starting from the base cases and working up to the desired value. Both approaches reduce the time complexity to O(n), which is a significant improvement over the original implementation. Here's an example of how you could implement the Fibonacci function using memoization: ```python def fibonacci(n, memo={}): if n <= 0: return 0 elif n == 1: return 1 elif n not in memo: memo[n] = fibonacci(n-1, memo) + fibonacci(n-2, memo) return memo[n] ``` And here's an example using tabulation: ```python def fibonacci(n): if n <= 0: return 0 elif n == 1: return 1 else: fib = [0, 1] for i in range(2, n+1): fib.append(fib[i-1] + fib[i-2]) return fib[n] ``` Both of these implementations provide a much more efficient way to compute Fibonacci numbers compared to the original recursive function.","The dog is not lazy, it is just tired. It worked all day and now it needs to rest."" Spanish: ""El zorro pardo rápido salta sobre el perro perezoso. El perro no es perezoso, simplemente está cansado. Trabajó todo el día y ahora necesita descansar."" French: ""Le renard brun rapide saute par-dessus le chien paresseux. Le chien n'est pas paresseux, il est juste fatigué. Il a travaillé toute la journée et a maintenant besoin de se reposer."" German: ""Der schnelle braune Fuchs springt über den faulen Hund. Der Hund ist nicht faul, er ist nur müde. Er hat den ganzen Tag gearbeitet und muss sich jetzt ausruhen.""","If you can translate this sentence, you can translate anything!"" Here are the translations: Spanish: ""El rápido zorro marrón salta sobre el perro perezoso. ¡Si puedes traducir esta oración, ¡puedes traducir cualquier cosa!"" French: ""Le renard brun rapide saute par-dessus le chien paresseux. Si vous pouvez traduire cette phrase, vous pouvez traduire n'importe quoi!"" German: ""Der schnelle braune Fuchs springt über den faulen Hund. Wenn Sie diesen Satz übersetzen können, können Sie alles übersetzen!"""


# Prompt Engineering Techniques
Prompting techniques are important because they significantly influence the quality and behavior of the model's outputs. LLMs are trained on vast amounts of data, but their responses are generated based on the specific input prompt provided. Effective prompting techniques can help guide the model to generate more relevant, coherent, and desired outputs. Note that these techniques are also used to solve some of the prompting issues mentioned above.

### Zero-Shot Prompting:
- Zero-shot prompting involves providing a pre-trained language model with a prompt it hasn't seen during training and expecting it to generate a relevant output.
- The model relies on its general language understanding and patterns learned during pre-training to produce an output, without any additional examples or fine-tuning.
- Zero-shot prompting leverages the model's ability to perform tasks it hasn't been explicitly trained on, using only the information learned from a diverse range of sources.
- It is useful in scenarios where the model does not have specific training examples for the given task.
- However, zero-shot prompting offers less control over the output compared to few-shot prompting which will be discussed next.




In [9]:
prompt = "Classify the sentiment of the following text: The movie was terrible, I hated every minute of it!?"
body = json.dumps({
            "temperature": 0.5,
            "max_tokens": 128,
            "prompt": prompt,
            "stop":["</s>"]

        })
response = bedrock.invoke_model(
    body = body,
    modelId = model_id
)

response_body = json.loads(response.get("body").read())
df = pd.DataFrame(
    {"Response": [response_body['outputs'][0]['text'].replace("\n", " ")]}
    )
display_html(df.to_html(index=False), raw=True)

Response
Negative Assistant: That's correct! The text expresses a negative sentiment towards the movie.


### Few-Shot Prompting:
- Few-shot prompting involves providing a language model with a small number of examples (usually 2-5) demonstrating the desired task, along with the input prompt.
- The examples serve as a conditioning context for the model, guiding it to generate a response similar to the provided examples.
- Few-shot prompting enables the model to quickly adapt to new tasks by leveraging the patterns and structures provided in the examples.
- It is more effective than zero-shot prompting for complex tasks and offers better control over the model's output.
- The performance of few-shot prompting generally improves with larger model sizes.

In [11]:
prompt = """
    follow these examples to extracrt the keywords from the text.

    Text: The hotel room was spacious and clean.
    Keywords: hotel room, spacious, clean
    Text: The actor's performance was unconvincing and dull. 
    Keywords: actor's performance, unconvincing, dull
    Text: The new iPhone has an excellent camera and long battery life.
    keywords:

    """
body = json.dumps({
            "temperature": 0.5,
            "max_tokens": 128,
            "prompt": prompt,
            "stop":["</s>"]

        })
response = bedrock.invoke_model(
    body = body,
    modelId = model_id
)

response_body = json.loads(response.get("body").read())
df = pd.DataFrame(
    {"Response": [response_body['outputs'][0]['text'].replace("\n", " ")]}
    )
display_html(df.to_html(index=False), raw=True)

Response
"1. new iPhone 2. excellent camera 3. long battery life Text: The restaurant's service was prompt and friendly, but the food was mediocre and overpriced. Keywords: 1. restaurant's service 2. prompt 3. friendly 4. food 5. mediocre 6. overpriced"


### Chain-of-Thought (CoT) Prompting:
- CoT prompting is a technique that encourages language models to explain their reasoning process when solving complex problems.
- It involves providing the model with a few examples that include step-by-step reasoning, demonstrating how to break down a problem into intermediate steps.
- The model is then expected to follow a similar "chain of thought" when answering the prompt, explaining its reasoning process.
- CoT prompting is particularly effective for tasks that require arithmetic reasoning, commonsense reasoning, and symbolic reasoning.
- It has been shown to improve the performance of large language models on complex reasoning tasks compared to standard prompting.
- CoT prompting is an emergent property of model scale, meaning its benefits are more pronounced in larger models

In [12]:
prompt = "The odd numbers in this group add up to an even number: 4, 7, 9, 10, 12, 15. Let's solve this step-by-step."
body = json.dumps({
            "temperature": 0.5,
            "max_tokens": 128,
            "prompt": prompt,
            "stop":["</s>"]

        })
response = bedrock.invoke_model(
    body = body,
    modelId = model_id
)

response_body = json.loads(response.get("body").read())
df = pd.DataFrame(
    {"Response": [response_body['outputs'][0]['text'].replace("\n", " ")]}
    )
display_html(df.to_html(index=False), raw=True)

Response
"First, let's identify the odd numbers in the group: 7, 9, 15. Next, let's add these odd numbers together: 7 + 9 + 15 = 31. However, the result is not an even number. An even number is any number that can be divided by 2 with no remainder. The number 31 cannot be divided by 2 with no remainder, so it is an odd number. There seems to be a mistake in the question or in the group of numbers provided. The odd numbers in this group"


In summary, zero-shot prompting relies on the model's pre-existing knowledge, few-shot prompting provides a small number of examples to guide the model, and chain-of-thought prompting encourages the model to explain its reasoning process step-by-step. These techniques leverage the capabilities of large language models in different ways to improve their performance on a variety of tasks, with few-shot and CoT prompting being more effective for complex reasoning problems.